In [1]:
import torch
import numpy as np
import os, glob
import cv2 as cv

In [2]:
def local_normalize_image(img):
    gray=cv.cvtColor(img, cv.COLOG_RGB2GRAY)
    float_gray=gray.astype(np.float32)/255.0
    blur = cv.GaussianBlur(float_gray, (0, 0), sigmaX=2, sigmaY=2)
    num = float_gray - blur

    blur = cv.GaussianBlur(num*num, (0, 0), sigmaX=20, sigmaY=20)
    den = cv.pow(blur, 0.5)+0.0000001

    gray = num / den

    cv.normalize(gray, dst=gray, alpha=0.0, beta=1.0, norm_type=cv.NORM_MINMAX)

    gray = np.concatenate((gray[:,:,np.newaxis],gray[:,:,np.newaxis],gray[:,:,np.newaxis]),axis=2)

    return gray

In [ ]:
class DataLoader(object):
    """ Data loading class for training heatmap-attention-padding network

    Args:
        dataset_dir: Folder contain pkl files files
        batch_size: training batch size
        image_height, image_width: input image height and width
        opt: flags from input parser
    
    Returns:
        new_mask: A gauss smoothed tensor

    """
    def __init__(self, dataset_dir, batch_size, image_height, image_width, num_epochs, num_views):
        self.dataset_dir=dataset_dir
        self.batch_size=batch_size
        self.image_height=image_height
        self.image_width=image_width
        self.num_epochs = num_epochs
        self.num_views = num_views
        
        